In [1]:
%run import_libs.py

### get data

In [2]:
df_train = get_train_data(TRAIN_PATH='./data/train.parquet')
num_features = pd.read_csv("num_feats_after_filtering.csv")["0"].to_list()

df_train_agg = get_df_w_aggrs(df=df_train, feats=num_features)
df_train_target = get_target(TARGET_PATH='./data/train_labels.csv')
df_train = get_train_data_with_target_merged(df_train=df_train_agg, df_train_target=df_train_target)

(458913, 151)
(458913, 151)
(458913, 151)
(458913, 162)
(458913, 616)


In [3]:
df_test = get_test_data(TEST_PATH='./data/test.parquet')
df_test = get_df_w_aggrs(df=df_test, feats=num_features)

(924621, 151)
(924621, 151)
(924621, 151)
(924621, 162)
(924621, 616)


In [4]:
cat_features = [f"{f}_last" for f in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']]
cat_features

['B_30_last',
 'B_38_last',
 'D_114_last',
 'D_116_last',
 'D_117_last',
 'D_120_last',
 'D_126_last',
 'D_63_last',
 'D_64_last',
 'D_66_last',
 'D_68_last']

In [5]:
payment_feats = []
delinq_feats = []
spend_feats = []
balance_feats = []
risk_feats = []

for feat in list(df_train):
    if feat in cat_features:
        continue
    
    if feat[0] == 'P':
        #print(feat)
        payment_feats.append(feat)
    elif feat[0] == 'D':
        delinq_feats.append(feat)
    elif feat[0] == 'S':
        spend_feats.append(feat)
    elif feat[0] == 'B':
        balance_feats.append(feat)
    elif feat[0] == 'R':
        risk_feats.append(feat)

In [6]:
len(payment_feats) + len(delinq_feats) + len(spend_feats) + len(balance_feats) + len(risk_feats)

604

In [7]:
num_features = payment_feats + delinq_feats + spend_feats + balance_feats + risk_feats
len(num_features)

604

### load config

In [9]:
class CFG:
    DEBUG = False
    model = 'tabnet'
    N_folds = 5
    seed = 42
    batch_size = 512
    max_epochs = 60

In [12]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(seed = CFG.seed)

In [13]:
psutil.virtual_memory().percent

35.3

In [14]:
# using amex metric to evaluate tabnet
class Amex_tabnet(Metric):
    
  def __init__(self):
    self._name = 'amex_tabnet'
    self._maximize = True

  def __call__(self, y_true, y_pred):
    amex = get_amex_metric_calculated(y_true, y_pred[:, 1])
    return max(amex, 0.)

### Fit model

In [15]:
print('\n ', '-'*50)
print('\nTraining: ', CFG.model)
print('\n ', '-'*50)

print('\nSeed: ', CFG.seed)
print('N folds: ', CFG.N_folds)

print('\nN features: ', len(num_features))
print('\n')


  --------------------------------------------------

Training:  tabnet

  --------------------------------------------------

Seed:  42
N folds:  5

N features:  604




In [19]:
train = df_train.fillna(0)
test = df_test.fillna(0)

print('Shapes:', train.shape, test.shape)

Shapes: (458913, 617) (924621, 616)


In [21]:
# X_train = train.loc[train_idx]
# y_train = target.loc[train_idx]

# Create out of folds array
oof_predictions = np.zeros((train.shape[0]))
test_predictions = np.zeros(test.shape[0])
feature_importances = pd.DataFrame()
feature_importances["feature"] = train.columns.tolist()
stats = pd.DataFrame()
explain_matrices = []
masks_ =[]

target_col = 'target'
group_col = 'customer_ID'

target, groups = df_train[target_col].values, df_train[group_col].values
    
# kfold = StratifiedKFold(n_splits = CFG.N_folds, shuffle=True, random_state = CFG.seed)
sgkf = StratifiedGroupKFold(CFG.N_folds, shuffle=True, random_state=CFG.seed)

# for tr_idx, va_idx in sgkf.split(df_train[[group_col, target_col]], y, groups):

for fold, (tr_idx, va_idx) in enumerate(
                sgkf.split(df_train[[group_col, target_col]], target, groups)):
    print(f"Fold {fold}")

    ## DEBUG MODE
    if CFG.DEBUG == True:
        if fold > 0:
            print('\nDEBUG mode activated: Will train only one fold...\n')
            break      

    start = time.time()

    X_tr, X_va = df_train.iloc[tr_idx][num_features], df_train.iloc[va_idx][num_features]
    y_tr, y_va = target[tr_idx], target[va_idx]    

    # X_train, y_train = train.loc[train_idx], target.loc[train_idx]
    # X_valid, y_valid = train.loc[valid_idx], target.loc[valid_idx]        
        
    model = TabNetClassifier(n_d = 32,
                             n_a = 32,
                             n_steps = 3,
                             gamma = 1.3,
                             n_independent = 2,
                             n_shared = 2,
                             momentum = 0.02,
                             clip_value = None,
                             lambda_sparse = 1e-3,
                             optimizer_fn = torch.optim.Adam,
                             optimizer_params = dict(lr = 1e-3, weight_decay=1e-3),
                             scheduler_fn = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts,
                             scheduler_params = {'T_0':5,
                                                 'eta_min':1e-4,
                                                 'T_mult':1,
                                                 'last_epoch':-1},
                             mask_type = 'entmax',
                             seed = CFG.seed)
    
    ## train
    model.fit(np.array(X_tr),
              #np.array(y_tr.values.ravel()),
              y_tr,
              eval_set = [(np.array(X_va), y_va)],
              max_epochs = 3, # CFG.max_epochs
              patience = 50,
              batch_size = CFG.batch_size,
              eval_metric = ['auc', 'accuracy', Amex_tabnet]) # Last metric is used for early stopping
    
    # Saving best model
    saving_path_name = f"./fold{fold}"
    saved_filepath = model.save_model(saving_path_name)
    
    # model explanability
    explain_matrix, masks = model.explain(X_va.values)
    explain_matrices.append(explain_matrix)
    masks_.append(masks[0])
    masks_.append(masks[1])
    
    # Inference
    oof_predictions[va_idx] = model.predict_proba(X_va.values)[:, 1]
    
    #if CFG
    # logodds function
    
    test_predictions += model.predict_proba(test.values)[:, 1]/5
    feature_importances[f"importance_fold{fold}+1"] = model.feature_importances_
    
    # Loss , metric tracking
    stats[f'fold{fold+1}_train_loss'] = model.history['loss']
    stats[f'fold{fold+1}_val_metric'] = model.history['val_0_amex_tabnet']


    end = time.time()
    time_delta = np.round((end - start)/60, 2)
     
    print(f'\nFold {fold+1}/{CFG.N_folds} | {time_delta:.2f} min')

    ### free memory
    del X_train, y_train
    del X_valid, y_valid
    gc.collect()

print(f'OOF score across folds: {get_amex_metric_calculated(target, oof_predictions.flatten())}')

Fold 0


c:\Users\oobur\Projects\spbu_master\sem4\vkr\.venv\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.51859 | val_0_auc: 0.91092 | val_0_accuracy: 0.83555 | val_0_amex_tabnet: 0.5967  |  0:00:56s
epoch 1  | loss: 0.35551 | val_0_auc: 0.9277  | val_0_accuracy: 0.86246 | val_0_amex_tabnet: 0.6642  |  0:01:52s
epoch 2  | loss: 0.31299 | val_0_auc: 0.93591 | val_0_accuracy: 0.87087 | val_0_amex_tabnet: 0.68442 |  0:02:49s
Stop training because you reached max_epochs = 3 with best_epoch = 2 and best_val_0_amex_tabnet = 0.68442


c:\Users\oobur\Projects\spbu_master\sem4\vkr\.venv\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at ./fold0.zip


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object

In [ ]:
test_predictions

In [ ]:
INFERENCE = True

if INFERENCE:
    sub = pd.DataFrame({'customer_ID': df_test.customer_ID,
                        'prediction': np.mean(test_predictions, axis=0)})
    sub.to_csv('submission_lr_woe_features.csv', index=False)
    display(sub)